In [ ]:
# standard library imports
from ast import literal_eval
import itertools
import time
import re

# third-party imports
import numpy as np
import pandas as pd

# customisations
pd.set_option("max_columns", 100)

In [ ]:
raw_steamspy_data = pd.read_csv('steamspy_data.csv')

# print out number of rows and columns
print('Rows:', raw_steamspy_data.shape[0])
print('Columns:', raw_steamspy_data.shape[1])

raw_steamspy_data.head()

In [ ]:
raw_steamspy_data.isnull().sum()

In [ ]:
raw_steamspy_data['userscore'].value_counts().head()

In [ ]:
drop_cols = [
    'score_rank', # too many missing values
    'userscore', # too little variance (all have 0)
    'genre', 'developer', 'publisher', 'price' # provided by Steam data
    'average_2weeks', 'median_2weeks', 'ccu', 'owners' # not interested in temporally specific columns
            ]

In [ ]:
raw_steamspy_data['languages'].head()


In [ ]:
tags = raw_steamspy_data['tags']

print(tags[0])
tags.head()


In [ ]:
eval_row = literal_eval(tags[0])

print(eval_row['Action'])
print(eval_row['FPS'])

In [ ]:
values = [
    ['a', 'b'], # list
    ('b', 'c'), # tuple
    {'d': 'e'}  # dictionary
]

list(itertools.chain(*values))

In [ ]:
set(itertools.chain(*values))


In [ ]:
parsed_tags = tags.apply(lambda x: literal_eval(x))

cols = set(itertools.chain(*parsed_tags))

print('Number of unique tags:', len(cols))
print('\nFirst few tags:', sorted(list(cols))[:5])

In [ ]:
def parse_tags(x):
    x = literal_eval(x)
    
    if isinstance(x, dict):
        return x
    elif isinstance(x, list):
        return {}
    else:
        raise TypeError('Something other than dict or list found')

parsed_tags = tags.apply(parse_tags)
        
tag_data = pd.DataFrame()

for col in sorted(cols):
    # standardise column names
    col_name = col.lower().replace(' ', '_').replace('-', '_').replace("'", "")

    # check if column in row's dictionary of tags and return that value if it is, or 0 if it isn't
    tag_data[col_name] = parsed_tags.apply(lambda x: x[col] if col in x.keys() else 0)

tag_data.head()


In [ ]:
def parse_tags(x):
    x = literal_eval(x)

    if isinstance(x, dict):
        return ';'.join(list(x.keys())[:3])
    else:
        return np.nan
    
tags.apply(parse_tags).head()


In [ ]:



    #review rating = (positive # of reviews / total number of reviews) * 10    

raw_steamspy_data["total reviews"] = raw_steamspy_data["positive"] + raw_steamspy_data["negative"]

raw_steamspy_data["review_rating"] = (raw_steamspy_data["positive"] / raw_steamspy_data["total reviews"]) * 10 

    #rounds the rating to two decimal places

raw_steamspy_data["review_rating"] =  raw_steamspy_data["review_rating"].round(1)

    #converts float to str 

raw_steamspy_data["review_rating"] = raw_steamspy_data["review_rating"].astype(str)

raw_steamspy_data["review_rating"] = raw_steamspy_data["review_rating"]

raw_steamspy_data['review_rating'].head()


In [ ]:
def process_tags(df, export=False):
    if export: 
        
        tag_data = df[['appid', 'tags']].copy()
        
        def parse_export_tags(x):
            x = literal_eval(x)

            if isinstance(x, dict):
                return x
            elif isinstance(x, list):
                return {}
            else:
                raise TypeError('Something other than dict or list found')

        tag_data['tags'] = tag_data['tags'].apply(parse_export_tags)

        cols = set(itertools.chain(*tag_data['tags']))

        for col in sorted(cols):
            col_name = col.lower().replace(' ', '_').replace('-', '_').replace("'", "")

            tag_data[col_name] = tag_data['tags'].apply(lambda x: x[col] if col in x.keys() else 0)

        tag_data = tag_data.drop('tags', axis=1)

        tag_data.to_csv('steamspy_tag_data.csv', index=False)
        print("Exported tag data to 'steamspy_tag_data.csv'")
        
        
    def parse_tags(x):
        x = literal_eval(x)
        
        if isinstance(x, dict):
            return ';'.join(list(x.keys())[:3])
        else:
            return np.nan
    
    df['tags'] = df['tags'].apply(parse_tags)
    
    # rows with null tags seem to be superseded by newer release, so remove (e.g. dead island)
    df = df[df['tags'].notnull()]
    
    return df


def process(df):
    df = df.copy()
    
    # handle missing value
    df = df[(df['name'].notnull()) & (df['name'] != 'none')]
    df = df[df['developer'].notnull()]
    df = df[df['languages'].notnull()]
    df = df[df['price'].notnull()]
    
    # remove unwanted columns
    df = df.drop([
        'genre', 'developer', 'publisher', 'score_rank', 'userscore', 'average_2weeks',
        'median_2weeks', 'ccu', 'owners', 'price', 'initialprice', 'discount'
    ], axis=1)
    
    # keep top tags, exporting full tag data to file
    df = process_tags(df, export=True)
    
    return df


steamspy_data = process(raw_steamspy_data)
steamspy_data.head()

In [ ]:
steamspy_data.isnull().sum()

In [ ]:
steamspy_data.to_csv('steamspy_data.csv', index=False)


In [ ]:
steam_data = pd.read_csv('steam_data_clean.csv')
    
merged = steam_data.merge(steamspy_data, left_on='steam_appid', right_on='appid', suffixes=('', '_steamspy'))
merged.head()

In [ ]:
# remove overlapping columns
steam_clean = merged.drop(['name_steamspy', 'languages', 'steam_appid'], axis=1)

# reindex to reorder columns
steam_clean = steam_clean[[
    'appid',
    'name',
    'release_date',
    'english',
    'developer',
    'publisher',
    'platforms',
    'required_age',
    'categories',
    'genres',
    'tags',
    'positive',
    'negative',
    'total reviews',
    'review_rating',
    'average_forever',
    'median_forever',
    'price'
]]

steam_clean = steam_clean.rename({
    'positive': 'positive_ratings',
    'negative': 'negative_ratings',
    'average_forever': 'average_playtime',
    'median_forever': 'median_playtime'
}, axis=1)

steam_clean.head()


In [ ]:
# export clean dataset
steam_clean.to_csv('F:/VSCode Final/../Steam_Clean_Data.csv', index=False)